<a href="https://colab.research.google.com/github/aiguozhe01/DIC_Assignment/blob/master/Sprint_14_copied.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sprint ディープラーニングフレームワーク2

# 【問題1】公式チュートリアルモデルを分担して実行

* 選択課題：畳み込みニューラルネットワーク (Convolutional Neural Networks)
* 参照元url： https://colab.research.google.com/drive/1EjCLLDPX9vlTfT6alFIvyetG1OvmmD0Q?hl=ja#scrollTo=7Fv-UwP2S6gF

## TensorFlowのインポート

In [3]:
# !pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf
import pandas as pd
from tensorflow.keras import datasets, layers, models

## MNISTデータセットのダウンロードと準備

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# (枚数、縦pixel、横pixel、カラーチャンネル数)
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# ピクセルの値を 0~1の間に正規化
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
test_images.shape

(10000, 28, 28, 1)

## 畳み込みの基礎部分の作成

* Conv2DとMaxPooling2Dレイヤーのスタック
* CNNはbatchサイズを無視して、shape(image_height, image_width, color_channels)のテンソルをとる。
* 今回のMNIST画像はグレースケールのため、color_channels=1、RGBの場合は3
* 入力フォーマット(28, 28, 1)を引数input_shapeとして最初のレイヤーに渡す。

In [ ]:
model = models.Sequential()
# 最初の畳み込みレイヤー
# チャンネル数：32、抽出するストライド範囲、活性化関数、最初の層に必要な入力形式
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

# 最初の隠れ層
# プーリング演算：(2, 2)はダウンスケールする係数。今回はそれぞれの次元で半分。
model.add(layers.MaxPooling2D((2, 2)))

# 2層目
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# 2番目の隠れ層
model.add(layers.MaxPooling2D((2, 2)))

# 3層目
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [ ]:
# モデルのArchitectureを表示
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


## Denseレイヤーを追加

* 畳み込み後の出力（3, 3, 64）を、1つ以上のDenseレイヤーに入れて分類を実行する。
* Denseレイヤーは入力としてベクトル(1D)を取り入れる。
* そのため、畳み込み後の3D出力を1Dに平滑化してから、Denseレイヤーを追加する。
* 今回のMNISTは10個の出力クラスを持つため、最後のDenseレイヤーの出力を10にする。
* 最後にsoftmax関数を使用する。

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# モデルのArchitectureを再度表示する。
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

## モデルのコンパイルと学習

畳み込みニューラルネットワークの計算グラフが完成したので、それを用いて最適化、損失関数、測定基準をコンパイル化（どのような学習方法をさせるか）する。

* 最適化方法：adam
* 損失関数：sparse_categorical_crossentropy
* 測定基準：accuracy

epochs数は5回分で学習させる。

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1426 - accuracy: 0.9571
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0487 - accuracy: 0.9849
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0331 - accuracy: 0.9895
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0261 - accuracy: 0.9918
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0198 - accuracy: 0.9938


## 学習済みモデルの評価

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

313/313 - 1s - loss: 0.0329 - accuracy: 0.9901


In [ ]:
print(f'精度：{test_acc*100:.2f}%')

精度：99.01%


* 結果、99%以上のテスト精度を達成した。

# 【問題2】（アドバンス課題）様々な手法を実行

* 参照元url: https://github.com/tensorflow/models/tree/master/research/slim

# 【問題3】Iris（2値分類）をKerasで学習

* 隠れ層2枚、出力層1枚の多層パーセプトロン
* Kerasの基本的な流れ
  1. モデルを定義
  2. ネットワークを構築
  3. コンパイル
  4. 実行
----
**モデルを定義**
* 層の数：4
* 入力層：4変数
* 中間層（隠れ層①）：50ノード
* 中間層（隠れ層②）：100ノード
* 出力層（ラベル）：2変数（0 | 1)
* バッチ数：10
* epoch数：100（default=20）
* 活性化関数：relu
* 損失関数：categorical_crossentropy
* 最適化関数：Adam
* 評価関数：Accuracy（精度）

In [5]:
# ライブラリーとデータを読み出す。

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris


iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df = df.astype(float)
df['Species'] = iris.target
# 対象となる説明・目的変数（50～150行目）のみをスライスする。
df = df[50:150]

X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = df["Species"]

# 目的変数のラベルを0か1の二値に変換する。
y = y.replace(1, 0) # ラベル1を0に変換
y = y.replace(2, 1) # ラベル2を1に変換

X = np.array(X)
y = np.array(y)

# X = pd.DataFrame(iris.data[50:150, :], columns=iris.feature_names)
# y = pd.DataFrame(iris.target[50:150], columns=["Species"])

In [5]:
# 訓練データとテストデータへの分離
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
# 多層パーセプトロンの構築

model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(50, activation='relu', input_shape=(4, ), name="layer1"))
model.add(layers.Dense(100, activation='relu', name="layer2"))
model.add(layers.Dense(1, activation='softmax', name="output_layer"))

model.summary()

Model: "my_sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 50)                250       
_________________________________________________________________
layer2 (Dense)               (None, 100)               5100      
_________________________________________________________________
output_layer (Dense)         (None, 1)                 101       
Total params: 5,451
Trainable params: 5,451
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 構築したmodelを用いcompile（どのように学習させるか）を設定する。
from keras.optimizers import Adam
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
# 学習を実行する。
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=20,
                    validation_split=0.2
                    )

Epoch 1/20
6/6 [==============================] - 0s 21ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 2/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 3/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 4/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 5/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 6/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 7/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 

In [ ]:
model.evaluate(x = X_test, y = y_test)

1/1 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.4800


[0.0, 0.47999998927116394]

Test loss: 0.0
Test accuracy: 0.479

## 【問題4】Iris（多値分類）をKerasで学習

In [1]:
import time
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
from sklearn.preprocessing import OneHotEncoder

# データセットの読み込み
dataset_path ="/content/drive/My Drive/DIC/dataset/datasets_19_420_Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルをone-hotに変換
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit_transform(y[:, np.newaxis])
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [13]:
K.clear_session()

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation=tf.nn.relu, input_shape=(4,)))
model.add(tf.keras.layers.Dense(8, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 27        
Total params: 243
Trainable params: 243
Non-trainable params: 0
_________________________________________________________________


In [15]:
start = time.time()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=50,
                    verbose=2,
                    validation_data=(X_val, y_val))
end = time.time()
print('wall time: {:.4f}sec'.format(end-start))

score = model.evaluate(X_test, y_test, verbose=0)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
10/10 - 0s - loss: 1.3858 - accuracy: 0.4583 - val_loss: 1.0253 - val_accuracy: 0.7083
Epoch 2/50
10/10 - 0s - loss: 0.9359 - accuracy: 0.6875 - val_loss: 0.8029 - val_accuracy: 0.7083
Epoch 3/50
10/10 - 0s - loss: 0.7575 - accuracy: 0.6979 - val_loss: 0.6654 - val_accuracy: 0.7083
Epoch 4/50
10/10 - 0s - loss: 0.6003 - accuracy: 0.7188 - val_loss: 0.5113 - val_accuracy: 0.7083
Epoch 5/50
10/10 - 0s - loss: 0.4952 - accuracy: 0.6875 - val_loss: 0.4571 - val_accuracy: 0.7083
Epoch 6/50
10/10 - 0s - loss: 0.4408 - accuracy: 0.8021 - val_loss: 0.4359 - val_accuracy: 0.7917
Epoch 7/50
10/10 - 0s - loss: 0.3891 - accuracy: 0.8125 - val_loss: 0.3942 - val_accuracy: 0.7083
Epoch 8/50
10/10 - 0s - loss: 0.3423 - accuracy: 0.8958 - val_loss: 0.3649 - val_accuracy: 0.9167
Epoch 9/50
10/10 - 0s - loss: 0.3129 - accuracy: 0.9167 - val_loss: 0.3308 - val_accuracy: 0.9167
Epoch 10/50
10/10 - 0s - loss: 0.2499 - accuracy: 0.9479 - val_loss: 0.3818 - val_accuracy: 0.7500
Epoch 11/50
10/10 -

## 【問題5】House PricesをKerasで学習

In [17]:
from sklearn.preprocessing import StandardScaler

house_data = pd.read_csv('/content/drive/My Drive/DIC/dataset/House_Prices/house_prices_train.csv')

#GrLivArea、YearBuilt、SalePriceを抽出
train = house_data.loc[:, ['GrLivArea', 'YearBuilt', 'SalePrice']]

# DataFrameをndarrayに変換
X = np.array(train.iloc[:, :-1])
y = np.array(train.iloc[:, -1])[:,None]

# 特徴量を標準化
scaler = StandardScaler()
X = scaler.fit_transform(X)
# 目的変数を対数変換
y = np.log1p(y)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [18]:
K.clear_session()

input_data = tf.keras.layers.Input(shape=(2,))

l = tf.keras.layers.Dense(10, activation=tf.nn.relu)(input_data)
l = tf.keras.layers.Dense(10, activation=tf.nn.relu)(l)
output = tf.keras.layers.Dense(1, activation=tf.nn.relu)(l)

model = tf.keras.Model(input_data, output)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 10)                30        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


In [20]:
start = time.time()
model.compile(loss='mean_squared_error',
              optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.01),
              metrics=['mse'])

history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=50,
                    verbose=2,
                    validation_data=(X_val, y_val))
end = time.time()
print('wall time: {:.4f}sec'.format(end-start))

score = model.evaluate(X_test, y_test, verbose=0)
print('\nTest loss:', score[0])

Epoch 1/50
94/94 - 0s - loss: 37.9790 - mse: 37.9790 - val_loss: 2.2050 - val_mse: 2.2050
Epoch 2/50
94/94 - 0s - loss: 1.8309 - mse: 1.8309 - val_loss: 0.6570 - val_mse: 0.6570
Epoch 3/50
94/94 - 0s - loss: 0.5771 - mse: 0.5771 - val_loss: 0.2269 - val_mse: 0.2269
Epoch 4/50
94/94 - 0s - loss: 0.2706 - mse: 0.2706 - val_loss: 0.1151 - val_mse: 0.1151
Epoch 5/50
94/94 - 0s - loss: 0.1631 - mse: 0.1631 - val_loss: 0.0743 - val_mse: 0.0743
Epoch 6/50
94/94 - 0s - loss: 0.1057 - mse: 0.1057 - val_loss: 0.0734 - val_mse: 0.0734
Epoch 7/50
94/94 - 0s - loss: 0.0764 - mse: 0.0764 - val_loss: 0.0558 - val_mse: 0.0558
Epoch 8/50
94/94 - 0s - loss: 0.0655 - mse: 0.0655 - val_loss: 0.0464 - val_mse: 0.0464
Epoch 9/50
94/94 - 0s - loss: 0.0637 - mse: 0.0637 - val_loss: 0.0473 - val_mse: 0.0473
Epoch 10/50
94/94 - 0s - loss: 0.0598 - mse: 0.0598 - val_loss: 0.0569 - val_mse: 0.0569
Epoch 11/50
94/94 - 0s - loss: 0.0575 - mse: 0.0575 - val_loss: 0.0512 - val_mse: 0.0512
Epoch 12/50
94/94 - 0s - los

## 【問題6】MNISTをKerasで学習

In [7]:
#《データセットをダウンロードするコード》
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

X_train = X_train[:,:,:,None]  # NHWC
X_test = X_test[:,:,:,None]  # NHWC

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.transform(y_test[:, np.newaxis])

# trainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [8]:
K.clear_session()

input_data = tf.keras.layers.Input(shape=(28,28,1))  # channels last

l = tf.keras.layers.Conv2D(8, (3,3), padding='valid', activation=tf.nn.relu)(input_data)
l = tf.keras.layers.MaxPool2D((3,3), padding='same')(l)
l = tf.keras.layers.Conv2D(16, (3,3), padding='valid', activation=tf.nn.relu)(l)
l = tf.keras.layers.MaxPool2D((3,3), padding='same')(l)
l = tf.keras.layers.Flatten()(l)
output = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(l)

model = tf.keras.Model(input_data, output)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 8)         80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 9, 8)           0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 16)          1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 144)               0         
_________________________________________________________________
dense (Dense)                (None, 10)               

In [9]:
start = time.time()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.01),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=20,
                    epochs=20,
                    verbose=2,
                    validation_data=(X_val, y_val))
end = time.time()
print('wall time: {:.4f}sec'.format(end-start))

score = model.evaluate(X_test, y_test, verbose=0)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
2400/2400 - 19s - loss: 0.1774 - accuracy: 0.9438 - val_loss: 0.0944 - val_accuracy: 0.9698
Epoch 2/20
2400/2400 - 18s - loss: 0.0935 - accuracy: 0.9720 - val_loss: 0.1038 - val_accuracy: 0.9696
Epoch 3/20
2400/2400 - 18s - loss: 0.0879 - accuracy: 0.9729 - val_loss: 0.1052 - val_accuracy: 0.9685
Epoch 4/20
2400/2400 - 18s - loss: 0.0849 - accuracy: 0.9747 - val_loss: 0.1007 - val_accuracy: 0.9718
Epoch 5/20
2400/2400 - 17s - loss: 0.0822 - accuracy: 0.9754 - val_loss: 0.1028 - val_accuracy: 0.9732
Epoch 6/20
2400/2400 - 18s - loss: 0.0845 - accuracy: 0.9753 - val_loss: 0.0763 - val_accuracy: 0.9768
Epoch 7/20
2400/2400 - 18s - loss: 0.0799 - accuracy: 0.9766 - val_loss: 0.0818 - val_accuracy: 0.9787
Epoch 8/20
2400/2400 - 18s - loss: 0.0810 - accuracy: 0.9773 - val_loss: 0.0847 - val_accuracy: 0.9738
Epoch 9/20
2400/2400 - 19s - loss: 0.0801 - accuracy: 0.9759 - val_loss: 0.1061 - val_accuracy: 0.9703
Epoch 10/20
2400/2400 - 18s - loss: 0.0822 - accuracy: 0.9768 - val_loss: